# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## 1. Откройте и изучите файл

In [16]:
import pandas as pd
import joblib
from joblib import load
from joblib import dump
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [3]:
df.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


## 2. Разбейте данные на выборки

In [4]:
df_train1, df_valid= train_test_split(df, test_size = 0.20, random_state = 12345)

In [5]:
df_train, df_test = train_test_split(df_train1, test_size = 0.25, random_state = 12345)

Я разбил выборку в 2 этапа на 3 части - 60% обучающая выборка и 20-20 валидационная и тестовая

In [6]:
train_features = df_train.drop(['is_ultra'], axis = 1)
train_target = df_train['is_ultra']
test_features = df_test.drop(['is_ultra'], axis = 1)
test_target = df_test['is_ultra']
valid_features = df_valid.drop(['is_ultra'], axis = 1)
valid_target = df_valid['is_ultra']

## 3. Исследуйте модели

In [7]:
for i in range(1,16):    
    model_tree = DecisionTreeClassifier(random_state = 112325, max_depth = i)
    model_tree.fit(train_features, train_target)
    predictions_tree = model_tree.predict(valid_features)
    accuracy_tree = accuracy_score(predictions_tree, valid_target)
    print('max_depth :', i, '-', accuracy_tree)

max_depth : 1 - 0.7480559875583204
max_depth : 2 - 0.7838258164852255
max_depth : 3 - 0.7869362363919129
max_depth : 4 - 0.7869362363919129
max_depth : 5 - 0.7884914463452566
max_depth : 6 - 0.7807153965785381
max_depth : 7 - 0.7822706065318819
max_depth : 8 - 0.7776049766718507
max_depth : 9 - 0.7838258164852255
max_depth : 10 - 0.7729393468118196
max_depth : 11 - 0.7698289269051322
max_depth : 12 - 0.7651632970451011
max_depth : 13 - 0.7511664074650077
max_depth : 14 - 0.749611197511664
max_depth : 15 - 0.7527216174183515


Первая модель - дерево решений, в данной модели я меняю гиперпараметр - макимальная глубина, лучший результат с гулбиной 5 (0.788)

In [8]:
for i in range(10, 51, 5):
    model_forest = RandomForestClassifier(n_estimators = i, max_depth = 10, random_state = 12345)
    model_forest.fit(train_features, train_target)
    predictions_forest = model_forest.predict(valid_features)
    accuracy_forest = accuracy_score(predictions_forest, valid_target)
    print('n_estimators :', i, '-', accuracy_forest)

n_estimators : 10 - 0.7884914463452566
n_estimators : 15 - 0.7947122861586314
n_estimators : 20 - 0.7884914463452566
n_estimators : 25 - 0.7916018662519441
n_estimators : 30 - 0.7993779160186625
n_estimators : 35 - 0.7993779160186625
n_estimators : 40 - 0.8009331259720062
n_estimators : 45 - 0.7993779160186625
n_estimators : 50 - 0.7993779160186625


Вторая модель - Лес. сдесь я уже могу поменять 2 гиперпараметра, изначально я проверю параметр оценщик (estimators).
лучший результат получился при 40 оценщиках. (0.8)

In [20]:
for i in range(1, 20):
    model_forest = RandomForestClassifier(n_estimators = 40, max_depth = i, random_state = 12345)
    model_forest.fit(train_features, train_target)
    predictions_forest = model_forest.predict(valid_features)
    accuracy_forest = accuracy_score(predictions_forest, valid_target)
    print('max_depth :', i, '-', accuracy_forest)

max_depth : 1 - 0.7620528771384136
max_depth : 2 - 0.7838258164852255
max_depth : 3 - 0.7838258164852255
max_depth : 4 - 0.7900466562986003
max_depth : 5 - 0.7931570762052877
max_depth : 6 - 0.7931570762052877
max_depth : 7 - 0.7931570762052877
max_depth : 8 - 0.7962674961119751
max_depth : 9 - 0.7978227060653188
max_depth : 10 - 0.7993779160186625
max_depth : 11 - 0.7884914463452566
max_depth : 12 - 0.7978227060653188
max_depth : 13 - 0.7900466562986003
max_depth : 14 - 0.7869362363919129
max_depth : 15 - 0.7869362363919129
max_depth : 16 - 0.7853810264385692
max_depth : 17 - 0.7900466562986003
max_depth : 18 - 0.7838258164852255
max_depth : 19 - 0.7838258164852255


Подставив лучший результат estimators, я проверю глубину. Лучшая глубина получается - 10

In [11]:
    
model_regression = LogisticRegression(random_state=12345)
model_regression.fit(train_features, train_target)
predictions_regression = model_tree.predict(valid_features)
accuracy_regression = accuracy_score(predictions_tree, valid_target)
print(accuracy_regression)

0.7527216174183515


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


последняя модель - логистическая регрессия. У данной модели я не могу ничего поменять. результат получился нихе чем у других моделей хотя и достиг нужных 0.75

## 4. Проверьте модель на тестовой выборке

In [12]:

predictions_tree = model_tree.predict(test_features)
accuracy_tree = accuracy_score(predictions_tree, test_target)
print('Точность дерева на тестовой выборке :', accuracy_tree)

Точность дерева на тестовой выборке : 0.7465007776049767


In [13]:

predictions_forest = model_forest.predict(test_features)
accuracy_forest = accuracy_score(predictions_forest, test_target)
print('Точность леса на тестовой выборке :', accuracy_forest)

Точность леса на тестовой выборке : 0.7838258164852255


In [22]:

predictions_regression = model_tree.predict(test_features)
accuracy_regression = accuracy_score(predictions_tree, test_target)
print('Точность регрессии на тестовой выборке :', accuracy_regression)

Точность регрессии на тестовой выборке : 0.7465007776049767


## 5. (бонус) Проверьте модели на адекватность

In [19]:
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(valid_features, valid_target)
DummyClassifier(strategy='most_frequent')
dummy_clf.predict(test_features)
dummy_clf.score(test_features, test_target)

0.6889580093312597

Для проверки адекватоности я использую dummy classifier, что бы посмотреть случайные результаты (как если бы мы подбрасывали монету).

Результат получился ниже чем у всех моделей, соответсвенно они работают.

## 6. Вывод

На полученных данных я проверил 3 модели- Дерево, Лес, Регрессия. Наилучший результат показал лес, 0.8. как мне кажется одна из причин то что у леса я мог менять больше 1 гиперпараметра. Хучший результат показала логистическая регрессия 0.75.

 Далее, каждую модель я проверил на тестовой выборке. результаты получились меньше чем на валидационной. Причем больше всех потеряло дерево (0.4-0.5) остальные - (0.1-0.2)
 
 В конце я сравнил результаты тестевых испытайний с случаными результатами. Результаты моих моделей оказались выше случайных что гооврит о их адекватности.